#Installation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install emoji
!pip install simplejson
!pip install wmd
!pip install kneed
!pip install ray
!pip install ray[default]
!pip install -U ray

In [ ]:
# Packages to store and manipulate data
import pandas as pd
import numpy as np
# Example function using numpy:
from numpy import dot
from numpy.linalg import norm
import math
import os
import json
import simplejson as json
import csv
import string
import glob
import re
from datetime import datetime
pd.set_option('max_colwidth', 200)

# Plotting packages
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns 
import pickle
import math
import wmd
from scipy import spatial
from scipy.spatial import distance
from scipy.spatial.distance import pdist, squareform
from collections import Counter
import plotly.express as px
%matplotlib inline
import plotly.express as px
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# Package to clean text
import emoji
import re
from keras.preprocessing.text import Tokenizer
import operator
from operator import itemgetter
from operator import itemgetter
from tqdm import tqdm

# Package NPL
import nltk
nltk.download('punkt') #This is a library that helps us tokenize words and phrases
nltk.download('stopwords')
nltk.download ( 'wordnet' ) #We use the data and methods in this library to stem our data
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.corpus import brown
stop_words = stopwords.words('english')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk import bigrams
from nltk.corpus import brown
from nltk.util import ngrams
from nltk import word_tokenize
from nltk import RegexpParser

from textblob import TextBlob
import spacy
nlp = spacy.load('en_core_web_sm')
from scipy.sparse import csr_matrix, issparse  # , todense
import sys
from collections import defaultdict
import multiprocessing

# Gensim
import gensim
import gensim.downloader
from gensim.test.utils import common_texts
from gensim.models import Word2Vec
from gensim import models
from gensim import corpora
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.models.coherencemodel import CoherenceModel
import gensim.downloader as api
from gensim.test.utils import datapath
from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import Word2Vec
from gensim.parsing.preprocessing import remove_stopwords, STOPWORDS
from gensim.parsing.preprocessing import remove_stopwords, preprocess_string
from gensim.models.ldamulticore import LdaMulticore
import gensim

# Model building package: Sklearn
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from sklearn.datasets import make_multilabel_classification
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics import pairwise_distances
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.manifold import TSNE
from sklearn.datasets import load_digits
# Cluster documents: Sklearn
from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
# Metric distance: Sklearn

# T-SNE Clustering Chart 
# Get topic weights and dominant topics
from sklearn.manifold import TSNE
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook

#importing the glove library
#from glove import Corpus, Glove

3# Ray
import ray

#Importar e Filtrar o Dataframe dos Tweets


In [ ]:
#ray.init()
ray.shutdown()

In [ ]:
@ray.remote
class PreprocessingOperation:

  def __init__(self, csv_file_path):
    self.csv_file_path = csv_file_path
    self.dataframe = self.read_csv()
   
    self.__filter_columns()
    self.__filter_by_original_tweets()
    self.__convert_date()

  def read_csv(self):
    return pd.read_csv(self.csv_file_path, error_bad_lines=False, index_col=False, dtype='unicode')

  def __filter_columns(self):
    self.dataframe = self.dataframe.loc[:,['created_at','id','tweet']]
    self.dataframe.drop_duplicates()#inplace=True)
    self.dataframe.dropna(inplace=True)

  def __filter_by_original_tweets(self):
    self.dataframe = self.dataframe.loc[self.dataframe.tweet.apply(lambda x: x[:4]!='RT @')]

  def __convert_date(self):
    self.dataframe.drop(self.dataframe[ self.dataframe['created_at'] == 'created_at' ].index , inplace=True)
    self.dataframe['created_at'] = pd.to_datetime(self.dataframe['created_at']).dt.tz_localize(None) #fuso horário
    
  def get_dataframe(self):
    dataframe = self.dataframe
    del self.dataframe
    return dataframe

file_paths = [     
'Clima/data_22_02_12.csv',
'Comida/data_22_02_12.csv',
'Esporte/data_22_02_12.csv',
'Filmes e séries/data_22_02_12.csv',
'Jogos/data_22_02_12.csv',
'Música/data_22_02_12.csv',
'Política/data_22_02_12.csv',
'Religião/data_22_02_12.csv',
'Romance/data_22_02_12.csv',
'Saúde/data_22_02_12.csv']

# ray approach --> parallel
preprocessing_ops = [PreprocessingOperation.remote(file_path) for file_path in file_paths]
ray_dfs = [preprocessing_op.get_dataframe.remote() for preprocessing_op in preprocessing_ops]
dataframes = ray.get(ray_dfs)

In [ ]:
df = pd.DataFrame(columns=['created_at','id', 'tweet'])
for dataframe in dataframes:
  dataframe = pd.DataFrame(dataframe,columns=['created_at','id', 'tweet'])
  df = pd.concat([df, dataframe])
df = df.reset_index(drop=True)

#Pré-processamento dos Dados


In [ ]:
class Processing:
    
    def __init__(self, dataframe):
      self.dataframe = dataframe
      self.__remove_links()
      self.__remove_user()
      self.__remove_hashtag()
      self.__lower_tweets()
      self.__remove_numbers()
      self.__remove_punctuation()
      self.__remove_emoji()
      #self.__translate_emoji()
      self.__remove_short_words_and_stopwords()
      self.__convert_to_string()
      self.__lower_tweets()
      self.__remove_double_spacing()
      self.__remove_nan_in_clean_tweet()

    def __remove_links(self):
      new_tweet = []
      for tweet in self.dataframe.tweet:
          tweet = re.sub(r'http\S+', '', tweet) # Remove http links
          tweet = re.sub(r'bit.ly/\S+', '', tweet) # Remove bitly links
          tweet = re.sub(r'&amp', '', tweet) # Remove amp
          tweet = tweet.strip('[link]') # Remove [links]
          new_tweet.append(tweet)
      self.dataframe['clean_tweet'] = new_tweet 
    
    def __remove_user(self):
      new_tweet = []
      for tweet in self.dataframe.clean_tweet:
          tweet = re.sub(r'(@[A-Za-z0-9-_]+[A-Za-z0-9-_]+)', '', tweet) # Remove tweeted at
          new_tweet.append(tweet)
      self.dataframe['clean_tweet'] = new_tweet 

    def __remove_hashtag(self):
      new_tweet = []
      for tweet in self.dataframe.clean_tweet:
          tweet = re.sub(r'(#[A-Za-z0-9-_]+[A-Za-z0-9-_]+)', '', tweet) # Remove hashtags
          new_tweet.append(tweet)
      self.dataframe['clean_tweet'] = new_tweet 

    def __lower_tweets(self):
      self.dataframe['clean_tweet'] = self.dataframe['clean_tweet'].apply(lambda x: x.lower())  
        
    def __remove_punctuation(self):
      my_punctuation = '#!"…¨$%&\'”’(“)*+,-./:;<=>?[\\]^_‘`{|}~•@°'
      new_tweet = []
      for tweet in self.dataframe.clean_tweet:
          tweet = re.sub('['+my_punctuation+']+', ' ', tweet) 
          new_tweet.append(tweet)
      self.dataframe['clean_tweet'] = new_tweet

    def __remove_numbers(self):
      new_tweet = []
      for tweet in self.dataframe.clean_tweet:
          tweet = re.sub('([0-9]+)', '', tweet)
          tweet = tweet.strip()
          new_tweet.append(tweet)
      self.dataframe['clean_tweet'] = new_tweet
      
    def __translate_emoji(self):
        new_tweet = []
        for tweet in self.dataframe.clean_tweet:
            tweet = emoji.demojize(tweet) 
            tweet = re.sub(r':', ' ',  tweet)
            tweet = re.sub(r"[^a-zA-Z0-9-_]", ' ', tweet) 
            new_tweet.append(tweet)
        self.dataframe['clean_tweet'] = new_tweet

    def __remove_emoji(self):
      new_tweet = []
      for tweet in self.dataframe.clean_tweet:
        emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
        tweet = emoji_pattern.sub(r'', tweet)
        new_tweet.append(tweet)
      self.dataframe['clean_tweet'] = new_tweet

    def __remove_short_words_and_stopwords(self):
      all_stopwords = gensim.parsing.preprocessing.STOPWORDS
      new_tweet = []
      stop = stopwords.words('english')
      stop.extend(['est','be','on','!','at','.',':','...','@',',','#','will','.m','in','a','the','with','to','by','and','my','is',
                   'of','for','new','via','are','that','has','have','all','as','it','so','they','do','he','just','this',
                   'was','who','your','from','his','about','get','but','am','up','if','can','would','than','should','dont',
                   'had','or','were','did','there','got','even','its','an'])
      for tweet in self.dataframe.clean_tweet:
          tweet_len = [word for word in tweet.split(' ') if len(word)> 2 and (word not in stop and word not in all_stopwords)]
          new_tweet.append(tweet_len)
      self.dataframe['clean_tweet'] = new_tweet 

    def __convert_to_string(self):
      self.new_tweet = []
      for tweet_token_list in self.dataframe.clean_tweet:
          tweet = ' '.join(tweet_token_list)
          self.new_tweet.append(tweet)
      self.dataframe['clean_tweet'] = self.new_tweet

    def __remove_double_spacing(self):
      new_tweet = []
      for tweet in self.dataframe.clean_tweet:
          tweet = re.sub(r'\s+', ' ', tweet) 
          new_tweet.append(tweet)
      self.dataframe['clean_tweet'] = new_tweet

    def __remove_nan_in_clean_tweet(self):
      self.dataframe['clean_tweet'].replace('', np.nan, inplace=True)
      self.dataframe.dropna(subset=['clean_tweet'], inplace=True)
      self.dataframe['clean_tweet'].str.replace(' ', '')
      self.dataframe['clean_tweet'].str.strip()

    def get_new_dataframe(self):
      return self.dataframe

In [ ]:
tweet = Processing(df)
df_new = tweet.get_new_dataframe()
df_new.head(50)